In [11]:
minmaxInterval = 2000

## open port

In [12]:
!echo 'quzinee1!'| sudo -S chmod a+rw /dev/ttyUSB0

[sudo] password for terry: 

## BASIC USAGE OF XL320 CLASS

In [13]:
import rospy
from trajectory_msgs.msg import JointTrajectory, JointTrajectoryPoint

rospy.init_node("Experiment2")


class Soro(object):
    def __init__(self):
        self.transform_data_pub = rospy.Publisher('/dynamixel/control', JointTrajectory, queue_size=10)

    def actuate(self, data:JointTrajectory):
        self.transform_data_pub.publish(data)


soro = Soro()
rate = rospy.Rate(1) # 10 Hz

In [14]:
import sys
sys.path.append(".")
""" FOR DYNAMIXEL """
from model.class_xc330 import xc330
""" FOR MODERN DRIVER """
import roslib; roslib.load_manifest('ur_driver')
import rospy
import actionlib
from control_msgs.msg import *
from trajectory_msgs.msg import *
from sensor_msgs.msg import JointState

from class_vicon_marker import VICON
from class_vicon_base import ViconBase
import copy
import numpy as np

from numpy import pi as PI
DEG90 = PI/2

class UR(object):
    def __init__(self,):
        self.JOINT_NAMES = ['shoulder_pan_joint', 'shoulder_lift_joint', 'elbow_joint',
                            'wrist_1_joint', 'wrist_2_joint', 'wrist_3_joint']
        self.client      = None
        
   
    def move_arm_speed(self, traj:JointTrajectory, speed_limit):
        try: 
            g = FollowJointTrajectoryGoal()
            g.trajectory = copy.deepcopy(traj)
            g.trajectory.joint_names = self.JOINT_NAMES
            joint_states = rospy.wait_for_message("joint_states", JointState)
            joints_pos   = joint_states.position

            init_point = JointTrajectoryPoint()
            init_point.positions = joints_pos
            init_point.time_from_start = rospy.Duration.from_sec(0)
            init_point.velocities = [0 for _ in range(6)]
            g.trajectory.points.insert(0, init_point)

            q_list = []
            time_list = []
            for point in g.trajectory.points:
                q_list.append(point.positions)
                time_list.append(point.time_from_start.to_sec())

            print(q_list)

            for i in range(len(q_list)-1):
                q_before = np.array(q_list[i])
                q_after  = np.array(q_list[i+1])
                print(q_after, q_before)
                time_before = time_list[i]
                time_after = time_list[i+1]

                diff_q = q_after - q_before
                diff_time = time_after - time_before

                speed = np.linalg.norm(diff_q)/diff_time
                print(speed)
                assert speed <= speed_limit
            
            

            self.client.send_goal(g)
            # self.client.wait_for_result()
        except KeyboardInterrupt:
            self.client.cancel_goal()
            raise
        except:
            raise  

    def execute_arm_speed(self, traj, speed_limit):
        try:
            self.client = actionlib.SimpleActionClient('follow_joint_trajectory', FollowJointTrajectoryAction)
            print("Waiting for server...")
            self.client.wait_for_server()
            print("Connected to server")
            """ Initialize """
            self.move_arm_speed(traj, speed_limit)
            print("Finish plan")

        except KeyboardInterrupt:
            rospy.signal_shutdown("KeyboardInterrupt")
            raise      

    def execute_arm_time(self, joints, movetime):
        try:
            self.client = actionlib.SimpleActionClient('follow_joint_trajectory', FollowJointTrajectoryAction)
            print("Waiting for server...")
            self.client.wait_for_server()
            print("Connected to server")
            """ Initialize """
            self.move_arm_time(joints, movetime)
            print("Finish plan")

        except KeyboardInterrupt:
            rospy.signal_shutdown("KeyboardInterrupt")
            raise      


## INSTANTIATE Robot 

In [15]:
robot = UR()
init_joint =[0,-DEG90,DEG90,-DEG90,-DEG90, DEG90]

In [16]:
q =[0,-DEG90,DEG90,-DEG90,-DEG90, DEG90 * 0]

q_traj = JointTrajectory()

point = JointTrajectoryPoint()
point.positions = q
point.velocities = [0 for _ in range(6)]
point.time_from_start = rospy.Duration.from_sec(5)

q_traj.points.append(point)

robot.execute_arm_speed(q_traj, speed_limit=0.5)
# traj

Waiting for server...
Connected to server
[(-7.788287561538709e-07, -1.571214338342184, 1.5709283987628382, -1.5704820913127442, -1.5708096663104456, -1.4607106344044496e-05), [0, -1.5707963267948966, 1.5707963267948966, -1.5707963267948966, -1.5707963267948966, 0.0]]
[ 0.         -1.57079633  1.57079633 -1.57079633 -1.57079633  0.        ] [-7.78828756e-07 -1.57121434e+00  1.57092840e+00 -1.57048209e+00
 -1.57080967e+00 -1.46071063e-05]
0.00010794674861803948
Finish plan


In [17]:
motor_control = [0,0]

motor_traj = JointTrajectory()
point = JointTrajectoryPoint()
point.positions = motor_control
point.velocities = [0 for _ in range(2)]
point.time_from_start = rospy.Duration.from_sec(2)

motor_traj.points.append(point)
soro.actuate(motor_traj)

In [18]:
motor_array = np.load("planned_traj/circle/motor_array.npy")
qs_array = np.load("planned_traj/circle/qs_array.npy")

motor_array.shape, qs_array.shape

((20, 2), (20, 6))

In [19]:
qs_array[8], init_joint

(array([-0.02628223, -0.92816806,  1.3628775 , -2.0055072 , -1.570796  ,
        -0.02628358], dtype=float32),
 [0,
  -1.5707963267948966,
  1.5707963267948966,
  -1.5707963267948966,
  -1.5707963267948966,
  1.5707963267948966])

In [20]:
q_traj = JointTrajectory()
motor_traj = JointTrajectory()

unit_time = 3

for i,(qs, motor_control) in enumerate(zip(qs_array, motor_array)):
    point = JointTrajectoryPoint()
    point.positions = qs
    point.velocities = [0 for _ in range(6)]
    point.time_from_start = rospy.Duration.from_sec(unit_time * (i+1))
    q_traj.points.append(point)
    
    point = JointTrajectoryPoint()
    point.positions = motor_control
    point.velocities = [0 for _ in range(2)]
    point.time_from_start = rospy.Duration.from_sec(unit_time * (i+1))
    motor_traj.points.append(point)
    

In [21]:
robot.execute_arm_speed(q_traj, speed_limit=0.5)
soro.actuate(motor_traj)

Waiting for server...
Connected to server
[(-3.6541615621388246e-05, -1.5711547336974085, 1.5709283987628382, -1.570470170383789, -1.5708096663104456, -3.8448964254200746e-05), array([-0.07520548, -0.91235256,  1.3309155 , -1.9893613 , -1.5707959 ,
       -0.07520763], dtype=float32), array([-0.07506419, -0.9222058 ,  1.3494447 , -1.9980342 , -1.5707964 ,
       -0.07506334], dtype=float32), array([-0.07244632, -0.93489414,  1.3722702 , -2.008171  , -1.5707964 ,
       -0.07244546], dtype=float32), array([-0.06696826, -0.9442702 ,  1.3896253 , -2.0161505 , -1.5707964 ,
       -0.06696773], dtype=float32), array([-0.05918093, -0.949543  ,  1.3999679 , -2.021221  , -1.5707964 ,
       -0.05918072], dtype=float32), array([-0.0503016 , -0.95002854,  1.4017054 , -2.022473  , -1.5707964 ,
       -0.05030172], dtype=float32), array([-0.04099071, -0.9462427 ,  1.3955001 , -2.020054  , -1.5707963 ,
       -0.04099114], dtype=float32), array([-0.03287306, -0.9386636 ,  1.3820105 , -2.0141442 , -